In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import math
import sklearn
from scipy import stats
import statsmodels.formula.api as sm

import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr
from sklearn import linear_model
import patsy
import statsmodels.formula.api as smf
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

%config InlineBackend.figure_format = 'png' #set 'png' here when working on notebook
%matplotlib inline

In [44]:
data = pd.read_excel("RawDataFinal1.xlsx")

In [45]:
train, test = train_test_split(data, test_size=0.2)#it is not time-series data
train.to_csv('train.csv')
test.to_csv('test.csv')

In [46]:
X = train.drop(columns=['SalePrice'])
Y = train['SalePrice']
alpha=np.arange(0,30,0.1)
singlefeatures = list(X)
lasso = linear_model.LassoCV(alphas=alpha,cv =5).fit(X,Y)
lassofeatures = np.array(singlefeatures)[np.abs(lasso.coef_)>1e-10]
print lassofeatures

[u'LotFrontage' u'LotArea' u'OverallQual' u'OverallCond' u'YearBuilt'
 u'YearRemodAdd' u'MasVnrArea' u'ExterQual' u'ExterCond' u'BsmtQual'
 u'BsmtCond' u'BsmtFinSF1' u'BsmtFinSF2' u'BsmtUnfSF' u'TotalBsmtSF'
 u'OnestFlrSF' u'TwondFlrSF' u'LowQualFinSF' u'GrLivArea' u'BsmtFullBath'
 u'BsmtHalfBath' u'FullBath' u'HalfBath' u'BedroomAbvGr' u'KitchenAbvGr'
 u'TotRmsAbvGrd' u'Fireplaces' u'GarageCars' u'GarageArea' u'WoodDeckSF'
 u'OpenPorchSF' u'EnclosedPorch' u'threeSsnPorch' u'ScreenPorch'
 u'PoolArea' u'MoSold' u'YrSold' u'Alley_NoAlley' u'Alley_Pave'
 u'LotShape_IR2' u'LotShape_IR3' u'LandContour_Bnk' u'LandContour_HLS'
 u'LandContour_Low' u'LandContour_Lvl' u'LandSlope_Gtl'
 u'Neighborhood_Blmngtn' u'Neighborhood_BrDale' u'Neighborhood_CollgCr'
 u'Neighborhood_Crawfor' u'Neighborhood_Edwards' u'Neighborhood_Gilbert'
 u'Neighborhood_Greens' u'Neighborhood_GrnHill' u'Neighborhood_IDOTRR'
 u'Neighborhood_MeadowV' u'Neighborhood_Mitchel' u'Neighborhood_NAmes'
 u'Neighborhood_NPkVill' u'Ne

In [47]:
lm = smf.ols('SalePrice ~ '+' + '.join(lassofeatures), data=train).fit()
print 'Here is in-sample R-square:'+str(lm.rsquared)
test_x=test.drop(columns=['SalePrice'])
test_y=test['SalePrice']
rsq = 1. - np.sum((lm.predict(test_x)-test_y)**2)/np.sum((
    test_y.mean()-test_y)**2)
print 'Here is the out-of-sample R-square:'+str(rsq)

Here is in-sample R-square:0.8857552807
Here is the out-of-sample R-square:0.885238718916


In [48]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [49]:
rms_test = sqrt(mean_squared_error(test['SalePrice'], lm.predict(test)))
print "Root mean squared error of the prediction is ", rms_test

Root mean squared error of the prediction is  25333.5824249


# Log Transformation


In [50]:
train = train.set_index([list(range(len(train)))])
for i in range(len(train["SalePrice"])):
    train.loc[i,('SalePrice')] = math.log(train.loc[i,('SalePrice')])

In [51]:
X = train.drop(columns=['SalePrice'])
Y = train['SalePrice']
alpha=np.arange(0,30,0.1)
singlefeatures = list(X)
lasso = linear_model.LassoCV(alphas=alpha,cv =5).fit(X,Y)
lassofeatures = np.array(singlefeatures)[np.abs(lasso.coef_)>1e-10]
print lassofeatures

[u'LotFrontage' u'LotArea' u'OverallQual' u'OverallCond' u'YearBuilt'
 u'YearRemodAdd' u'MasVnrArea' u'ExterQual' u'ExterCond' u'BsmtQual'
 u'BsmtCond' u'BsmtFinSF1' u'BsmtFinSF2' u'BsmtUnfSF' u'TotalBsmtSF'
 u'OnestFlrSF' u'TwondFlrSF' u'LowQualFinSF' u'GrLivArea' u'BsmtFullBath'
 u'BsmtHalfBath' u'FullBath' u'HalfBath' u'BedroomAbvGr' u'KitchenAbvGr'
 u'TotRmsAbvGrd' u'Fireplaces' u'GarageCars' u'GarageArea' u'WoodDeckSF'
 u'OpenPorchSF' u'EnclosedPorch' u'threeSsnPorch' u'ScreenPorch'
 u'PoolArea' u'MoSold' u'YrSold' u'Street_Grvl' u'Alley_Grvl'
 u'Alley_NoAlley' u'Alley_Pave' u'LotShape_IR1' u'LotShape_IR2'
 u'LotShape_IR3' u'LotShape_Reg' u'LandContour_Bnk' u'LandContour_HLS'
 u'LandContour_Low' u'LandContour_Lvl' u'Utilities_AllPub'
 u'Utilities_NoSeWa' u'Utilities_NoSewr' u'LandSlope_Gtl' u'LandSlope_Mod'
 u'LandSlope_Sev' u'Neighborhood_Blmngtn' u'Neighborhood_Blueste'
 u'Neighborhood_BrDale' u'Neighborhood_BrkSide' u'Neighborhood_ClearCr'
 u'Neighborhood_CollgCr' u'Neighborhoo

In [52]:
test = test.set_index([list(range(len(test)))])
for i in range(len(test["SalePrice"])):
    test.loc[i,('SalePrice')] = math.log(test.loc[i,('SalePrice')])

In [53]:
lm = smf.ols('SalePrice ~ '+' + '.join(lassofeatures), data=train).fit()
print 'Here is in-sample R-square:'+str(lm.rsquared)
test_x=test.drop(columns=['SalePrice'])
test_y=test['SalePrice']
rsq = 1. - np.sum((lm.predict(test_x)-test_y)**2)/np.sum((
    test_y.mean()-test_y)**2)
print 'Here is the out-of-sample R-square:'+str(rsq)

Here is in-sample R-square:0.912561649859
Here is the out-of-sample R-square:0.891487156425


In [54]:
for i in range(len(test["SalePrice"])):
    test.loc[i,('SalePrice')] = math.exp(test.loc[i,('SalePrice')])

In [55]:
prediction = lm.predict(test_x)
for i in range(len(prediction)):
      prediction[i]=math.exp(prediction[i])

In [56]:
rms_test = sqrt(mean_squared_error(test['SalePrice'], prediction))
print "Root mean squared error of the prediction is ", rms_test

Root mean squared error of the prediction is  22119.1886228


# BASELINE MODEL

In [57]:
singlefeatures=train.drop(columns=['SalePrice'])

In [58]:
for i in range(len(train["SalePrice"])):
    train.loc[i,('SalePrice')] = math.exp(train.loc[i,('SalePrice')])

In [59]:
lm = smf.ols('SalePrice ~ '+' + '.join(singlefeatures), data=train).fit()
print 'Here is in-sample R-square:'+str(lm.rsquared)
rsq = 1. - np.sum((lm.predict(test_x)-test_y)**2)/np.sum((
    test_y.mean()-test_y)**2)
print 'Here is the out-of-sample R-square:'+str(rsq)

Here is in-sample R-square:0.88657019625
Here is the out-of-sample R-square:0.883791424551


In [60]:
rms_test = sqrt(mean_squared_error(test['SalePrice'], lm.predict(test_x)))
print "Root mean squared error of the prediction is ", rms_test

Root mean squared error of the prediction is  25492.8272274
